# Using the BTagging Tool

Getting a tool to work properly is a little tricky. We'll do that here, and then move it into the hints file.

In [1]:
from func_adl_servicex_xaodr25 import FuncADLQueryPHYSLITE
from func_adl_servicex_xaodr25.xAOD.jet_v1 import Jet_v1
from servicex_analysis_utils import to_awk
from servicex import ServiceXSpec, Sample, dataset, deliver
from func_adl import ObjectStream, func_adl_callable
import ast
from typing import Tuple, TypeVar
# from servicex_local import deliver, LocalXAODCodegen, DockerScienceImage, SXLocalAdaptor, WSL2ScienceImage

Setup the code properly!

In [2]:
physlite = FuncADLQueryPHYSLITE()

Now configure the tool to run

- Taking examples from [use of the JetSelectionTool in the data extractor](https://github.com/gordonwatts/sx_training_fetch/blob/main/calratio_training_data/cpp_xaod_utils.py#L123)
- BTaggingSelectionTool info from the [BTagging Docs](https://ftag.docs.cern.ch/calibrations/cdi/interface/btaggingefficiencytool/).
- And from some [example code](https://gitlab.cern.ch/atlas/athena/-/blob/main/PhysicsAnalysis/JetTagging/JetTagPerformanceCalibration/xAODBTaggingEfficiency/util/BTaggingSelectionToolTester.cxx).
- And a better [working example](https://gitlab.cern.ch/atlas/athena/-/blob/main/PhysicsAnalysis/JetTagging/JetTagPerformanceCalibration/xAODBTaggingEfficiency/util/BTaggingToolsExample.cxx)

In [3]:
from dataclasses import dataclass
from typing import Callable, Iterable, List, Optional

T = TypeVar("T")


@dataclass
class ToolInfo:
    name: str


def make_a_tool(
    query: ObjectStream[T],
    tool_name: str,
    tool_type: str,
    include_files: Optional[List[str]],
    init_lines: List[str] = [],
) -> Tuple[ObjectStream[T], ToolInfo]:
    """
    Injects C++ code into the query to initialize a tool of the specified type.

    This function sets up the necessary C++ code to create and initialize a tool (such as
    BTaggingSelectionTool) in the analysis workflow. The tool will be available in the C++
    code under the variable name specified by `tool_name`, which can be referenced in
    initialization lines and later code.

    Args:
        query: The ObjectStream to attach the tool initialization metadata to.
        tool_name: The variable name to use for the tool instance in the C++ code.
        tool_type: The C++ class name of the tool to instantiate.
        include_files: List of C++ header files to include for the tool.
        init_lines: List of C++ code lines to run for tool initialization. You can use
            `{tool_name}` in these lines to refer to the tool variable. You should
            include the call to `ANA_CHECK({tool_name}->initialize());`.

    Returns:
        A tuple containing:
            - The updated ObjectStream with the tool initialization metadata.
            - A ToolInfo object containing the tool's name. Pass this to `make_tool_accessor`
    """
    # Define the C++ for the tool initialization

    query_base = query.MetaData(
        {
            "metadata_type": "inject_code",
            "name": tool_name,
            "header_includes": include_files,
            "private_members": [f"{tool_type} *{tool_name};"],
            "instance_initialization": [
                f'{tool_name}(new {tool_type} ("{tool_name}"))'
            ],
            "initialize_lines": [l.format(tool_name=tool_name) for l in init_lines],
            "link_libraries": ["xAODBTaggingEfficiencyLib"],
        }
    )

    return query_base, ToolInfo(name=tool_name)


def make_tool_accessor(
    t_info: ToolInfo,
    function_name: str,
    source_code: List[str],
    arguments: Iterable[Tuple[str, type]],
    return_type_cpp: str,
    return_type_python: str
):
    """
    Creates a Python-callable accessor for a C++ tool in the func_adl query.

    This function generates a Python function that, when called in a func_adl query,
    injects C++ code to call a method or function on a C++ tool instance (such as
    BTaggingSelectionTool). The accessor function can be used in the query to access
    tool functionality as if it were a regular Python function.

    Args:
        t_info: ToolInfo object containing the tool's variable name.
        function_name: Name of the accessor function (used in C++ and Python).
        source_code: List of C++ code lines to execute for the accessor. You can use
            `{tool_name}` in these lines to refer to the tool variable.
        arguments: Iterable of (argument_name, type) tuples specifying the arguments
            for the accessor function.
        return_type_cpp: The C++ return type of the accessor function.
        return_type_python: The Python return type annotation as a string.

    Returns:
        A Python function that can be used in a func_adl query to access the tool.
        NOTE: YOU MUST use the same name as `function_name` to store this:

            `my_name = make_tool_accessor(.., function_name="my_name", ...)`
    """
    # Define the callback function that `func_adl` will use to inject the calling code.
    def tool_callback(
        s: ObjectStream[T], a: ast.Call
    ) -> Tuple[ObjectStream[T], ast.Call]:
        new_s = s.MetaData(
            {
                "metadata_type": "add_cpp_function",
                "name": function_name,
                "code": [
                    "double result;",
                    *[l.format(tool_name=t_info.name) for l in source_code],
                ],
                "result": "result",
                "include_files": [],
                "arguments": [a[0] for a in arguments],
                "return_type": return_type_cpp,
            }
        )
        return new_s, a

    # Build a function type-shed that tells `func_adl` what the function signature is.
    # This is used to generate the correct C++ code for the function.
    def tool_call(**arg_dict):
        """
        NOTE: This is a dummy function that injects C++ into the object stream to do the
        actual work.
        """
        ...
    tool_call.__name__ = function_name
    tool_call.__annotations__['return'] = eval(return_type_python)

    return func_adl_callable(tool_callback)(tool_call)

Define the b-tagging selection tool, and the two ways we will access it.

In [4]:
query_base, tag_tool_info = make_a_tool(
    physlite,
    "btag_discriminator",
    "BTaggingSelectionTool",
    include_files=["xAODBTaggingEfficiency/BTaggingSelectionTool.h"],
    init_lines=[
        'ANA_CHECK(asg::setProperty({tool_name}, "OperatingPoint", "FixedCutBEff_77"));',
        "ANA_CHECK({tool_name}->initialize());",
    ],
)

tag_weight = make_tool_accessor(
    tag_tool_info,
    function_name="tag_weight",
    source_code=["ANA_CHECK({tool_name}->getTaggerWeight(*jet, result, false));"],
    arguments=[("jet", Jet_v1)],
    return_type_cpp="double",
    return_type_python="float",
)

jet_is_tagged = make_tool_accessor(
    tag_tool_info,
    function_name="jet_is_tagged",
    source_code=[
        "result = static_cast<bool>({tool_name}->accept(*jet));"
    ],
    arguments=[("jet", Jet_v1)],
    return_type_cpp="bool",
    return_type_python="bool",
)

Some other info:

* `13p6TeV/MC23_2025-06-17_GN2v01_v4.root` for Run 3 (MC23 files)
* `13TeV/MC20_2025-06-17_GN2v01_v4.root` for Run 2 (MC20 files)

It seems that we should not specify the btagging file - use whatever is default in the release.

Finally, we can actually execute the query!

In [5]:
query = (
    query_base.SelectMany(lambda event: event.Jets())
    .Where(lambda j: j.pt() > 20000 and abs(j.eta()) < 2)
    .Select(
        lambda j: {
            "is_tagged": jet_is_tagged(j),
            "weight": tag_weight(j),
            "pt": j.pt(),
            "eta": j.eta(),
        }
    )
)

And now we can run on a data sample!

In [6]:
ds_name = "mc23_13p6TeV:mc23_13p6TeV.601237.PhPy8EG_A14_ttbar_hdamp258p75_allhad.deriv.DAOD_PHYSLITE.e8514_s4369_r16083_p6697"
data = to_awk(
    deliver(
        ServiceXSpec(
            Sample=[
                Sample(
                    Name="ttbar_trijet",
                    Dataset=dataset.Rucio(ds_name),
                    NFiles=1,
                    Query=query,  # type: ignore
                )
            ]
        ),
    )
)
jets = data["ttbar_trijet"]

Output()

In [7]:
jets

<Array [{is_tagged: True, ...}, ..., {...}] type='323590 * {is_tagged: bool...'>